In [20]:
import torch
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from fedlern.models.mlp import MLP
from fedlern.models.resnet import ResNet18
from fedlern.train_utils import *
import fedlern.utils as utils
import torch.nn.functional as F

In [21]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#stats = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
stats = (0.49139968, 0.48215841, 0.44653091), (0.24703223, 0.24348513, 0.26158784)
batch_size = 100

In [22]:
train_loader, test_loader = prepare_dataloader_cifar(num_workers=8, train_batch_size=batch_size, eval_batch_size=batch_size, stats=stats)


Files already downloaded and verified


# RESNET no quantization

In [30]:
# Test the model

model = ResNet18()
load_model(model, './saved_models/resnet18_cifar10_92-5.ckpt', device)

loss, acc = utils.evaluate_model(model, test_loader, device,)
print(f'Loss: {loss}, Accuracy: {acc*100}%')
utils.print_model_size(model)


Loss: 0.0, Accuracy: 92.49999237060547%
44.77 MB


In [31]:
# Load the model
model = ResNet18()
load_model(model, './saved_models/resnet18_cifar10_92-2.pt', device)

# Test the model
loss, acc = utils.evaluate_model(model, test_loader, device,)
print(f'Loss: {loss}, Accuracy: {acc*100}%')
utils.print_model_size(model)

Loss: 0.0, Accuracy: 92.24999237060547%
44.77 MB


# RESNET Quantization 4 bits

In [17]:
# Test the model
load_dict = torch.load('./saved_models/resnet_4bits.pth')
#print(load_dict.keys())
state_dict = load_dict['state']
#print(state_dict.keys())
model = resnet20()
model.load_state_dict(state_dict)

loss, acc = utils.evaluate_model(model, test_loader, device)
print(f'Loss: {loss}, Accuracy: {acc*100}%')
utils.print_model_size(model)

Loss: 0.0, Accuracy: 68.44999694824219%
1.12 MB


# Quantization 8 bits

In [18]:
# Test the model
load_dict = torch.load('./saved_models/resnet_2bits.pth')
#print(load_dict.keys())
state_dict = load_dict['state']
#print(state_dict.keys())
model = resnet20()
model.load_state_dict(state_dict)

loss, acc = utils.evaluate_model(model, test_loader, device)
print(f'Loss: {loss}, Accuracy: {acc*100}%')
utils.print_model_size(model)

Loss: 0.0, Accuracy: 53.21999740600586%
1.12 MB


In [19]:
from tqdm.notebook import tqdm
from IPython.display import clear_output
# from tqdm.auto import tqdm  # notebook compatible
import time
bar = tqdm(range(5))
bar.refresh()
for i1 in bar:
    
    bar.set_description(f"Training {i1}",refresh=True)
    for i2 in tqdm(range(10), leave=False):
        # do something, e.g. sleep
        bar.write(f"i2={i2}")
        time.sleep(0.5)
    bar.set_description(f"Testing {i1}",refresh=True)
    for i3 in tqdm(range(10), leave=False):
        # do something, e.g. sleep
        bar.write(f"i3={i3}")
        time.sleep(0.5)
    clear_output(wait=True)

  0%|          | 0/10 [00:00<?, ?it/s]

i2=0
i2=1
i2=2
i2=3
i2=4
i2=5
i2=6
i2=7
i2=8
i2=9


  0%|          | 0/10 [00:00<?, ?it/s]

i3=0
i3=1
i3=2
i3=3
i3=4
i3=5
i3=6
i3=7
i3=8
i3=9


In [20]:
param = [ 28 * 28, # input
                512, 256, 128, 64,
                10 ] #output

transform = transforms.Compose([
    transforms.ToTensor(), # convert the image to a PyTorch tensor
    transforms.Normalize((0.5,), (0.5,)) # normalize the image with mean=0.5 and std=0.5
])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test_dataset = datasets.MNIST(root='data/', train=False, transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)



In [21]:
# Loading pretrained model
modeldict = torch.load('saved_models/mlp.ckpt')
model = MLP(param)
model.load_state_dict(modeldict)

q_dict = torch.load('saved_models/mlp_dynamicq.ckpt')
print(q_dict.keys())
model_qd = MLP(param)
model_qd.load_state_dict(q_dict)

loss, acc = utils.evaluate_model(model, test_loader, device)
print(f'Loss: {loss}, Accuracy: {acc*100}%')

# model.to(device)
# quantized_model.eval()
# model.eval()
# with torch.no_grad():
#     correctq = 0
#     totalq = 0
#     total = 0
#     correct = 0
#     for images, labels in test_loader:
#         images_cuda = images.to(device)
#         labels_cuda = labels.to(device)

#         outputsq = quantized_model(images)
#         _, predictedq = torch.max(outputsq.data, 1)
#         totalq += labels.size(0)
#         correctq += (predictedq == labels).sum().item()
        
#         outputs = model(images_cuda)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels_cuda.size(0)
#         correct += (predicted == labels_cuda).sum().item()
        

#     print('Accuracy of the quantized model on the test images: {} %'.format(100 * correctq / totalq))
#     print('Accuracy of the model on the test images: {} %'.format(100 * correct / total))




utils.print_model_size(model)
utils.print_model_size(model_qd)

odict_keys(['linears.0.scale', 'linears.0.zero_point', 'linears.0._packed_params.dtype', 'linears.0._packed_params._packed_params', 'linears.1.scale', 'linears.1.zero_point', 'linears.1._packed_params.dtype', 'linears.1._packed_params._packed_params', 'linears.2.scale', 'linears.2.zero_point', 'linears.2._packed_params.dtype', 'linears.2._packed_params._packed_params', 'linears.3.scale', 'linears.3.zero_point', 'linears.3._packed_params.dtype', 'linears.3._packed_params._packed_params', 'linears.4.scale', 'linears.4.zero_point', 'linears.4._packed_params.dtype', 'linears.4._packed_params._packed_params'])


/home/isaac/.local/lib/python3.8/site-packages/torch/_utils.py:314: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  device=storage.device,


RuntimeError: Error(s) in loading state_dict for MLP:
	Missing key(s) in state_dict: "linears.0.weight", "linears.0.bias", "linears.1.weight", "linears.1.bias", "linears.2.weight", "linears.2.bias", "linears.3.weight", "linears.3.bias", "linears.4.weight", "linears.4.bias". 
	Unexpected key(s) in state_dict: "linears.0.scale", "linears.0.zero_point", "linears.0._packed_params.dtype", "linears.0._packed_params._packed_params", "linears.1.scale", "linears.1.zero_point", "linears.1._packed_params.dtype", "linears.1._packed_params._packed_params", "linears.2.scale", "linears.2.zero_point", "linears.2._packed_params.dtype", "linears.2._packed_params._packed_params", "linears.3.scale", "linears.3.zero_point", "linears.3._packed_params.dtype", "linears.3._packed_params._packed_params", "linears.4.scale", "linears.4.zero_point", "linears.4._packed_params.dtype", "linears.4._packed_params._packed_params". 

In [2]:
import datetime

now = datetime.datetime.now()
now.strftime("%Y-%m-%d_%H-%M")

Current date and time : 
2023-06-16_18-50
